In [10]:
from PRClasses import DiscreteD, GaussD, HMM, MarkovChain
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import sounddevice as sd

from matplotlib import pyplot as plt
import numpy as np
import os

In [2]:
def GetSpeechFeatures(signal, fs, winlen, winstep, nfilt, nfft, winfunc):
    '''
    :param signal:
    :param fs:
    :param winlen:
    :param winstep:
    :param nfilt:
    :param nfft:
    :param winfunc:
    :return: the normalized MFCCs
    '''
    features_mfcc = mfcc(signal, fs,
                         winlen=winlen, winstep=winstep,
                         nfilt=nfilt,
                         nfft=nfft,
                         winfunc=winfunc)

    features_mfcc = features_mfcc.T
    features_mfcc_nor = np.zeros((features_mfcc.shape[0], features_mfcc.shape[1]))
    for i in range(features_mfcc.shape[0]):
        features_mfcc_nor[i, :] = features_mfcc[i, :] - np.mean(features_mfcc[i, :])
        features_mfcc_nor[i, :] = features_mfcc_nor[i, :] / np.std(features_mfcc_nor[i, :])

    return features_mfcc_nor

In [13]:
para_dir = 'para0525'
para_list = os.listdir(para_dir)
para_list.sort()
para_dir = para_dir + '/'

numf = 2    # num of fold
paras = [[] for i in range(numf)]
for i in range(numf):
    paras[i] = [para_list[i], para_list[i+numf], para_list[i+2*numf], para_list[i+3*numf], para_list[i+4*numf], para_list[i+5*numf], para_list[i+6*numf], para_list[i+7*numf], para_list[i+8*numf], para_list[i+9*numf]]
# print(paras[0]) # down_0.npy go_0.npy

ad_dir = 'train_audio'
ad_list = os.listdir(ad_dir)
ad_list.sort()
ad_dir = ad_dir + '/'

ad_err = np.zeros(10)
ad_err_var = np.zeros(10)
ad_err_cross = np.zeros((10, 10))

w = 0                                       # ad_count
for ad in ad_list:

    print('audio:', ad)
    wav_dir = os.path.join(ad_dir, ad)
    wav_list = os.listdir(wav_dir)
    # wav_list.sort()

    # # 5-Fold Cross Validation
    # a = len(wav_list) - 1920
    # test_list = [[] for i in range(5)]
    # test_list[0] = wav_list[0: a]
    # test_list[1] = wav_list[a: 2*a]
    # test_list[2] = wav_list[2*a: 3*a]
    # test_list[3] = wav_list[3*a: 4*a]
    # test_list[4] = wav_list[1920: ]

    # 2-Fold Cross Validation
    test_list = [[] for i in range(numf)]
    test_list[0] = wav_list[0: 600]
    test_list[1] = wav_list[1200: 1800]

    fold_err = np.zeros(numf)
    fold_err_cross = np.zeros((numf, 10))
    for tt in range(numf):                     # fold_count

        wav_dir = wav_dir + '/'
        wav0path = os.path.join(wav_dir, test_list[tt][0])
        fs, sig_0 = wav.read(wav0path)

        count = 0
        correct = 0
        error = 0
        err_cross = np.zeros(10)
        for wavf in test_list[tt]:

            wav_path = os.path.join(wav_dir, wavf)   # The path of each wav clip
            _, sig_temp = wav.read(wav_path)
            if len(sig_temp) == len(sig_0):
                # Calculating the MFCCs of each training audio
                mfcc_test = GetSpeechFeatures(sig_temp, fs, 0.02, 0.02, 30, 320, np.hanning)
                wav_prob = np.zeros(10)

                wp = 0
                for para in paras[tt]:

                    para_path = os.path.join(para_dir, para)
                    para = np.load(para_path, allow_pickle=True)
                    q = para[0]
                    A= para[1]
                    sD = para[2]
                    mc = MarkovChain(q, A)
                    h = HMM(mc, sD)

                    _, px, _ = h.Get_px(mfcc_test)
                    ifinf = np.isinf(px)
                    ifnan = np.isnan(px)
                    test_ovfl = np.count_nonzero(ifinf) + np.count_nonzero(ifnan)
                    # If overflowed, skip this clip sample
                    if test_ovfl > 0:
                        c = np.float('-inf')
                    # If not overflowed, train the HMM with the current clip
                    else:
                        c = h.logprob(mfcc_test)
                    wav_prob[wp] = c
                    wp = wp + 1

                pre = np.argmax(wav_prob)
                if pre == w:
                    count = count + 1
                    correct = correct + 1
                else:
                    count = count + 1
                    error = error + 1
                    err_cross[pre] = err_cross[pre] + 1

        fold_err[tt] = error / count
        fold_err_cross[tt] = err_cross / count
        print('Fold error rate_{}: '.format(tt), fold_err[tt])
        print('Fold error cross rate_{}: '.format(tt), fold_err_cross[tt])

    print('Results of one audio:')
    print('{}-Fold error rate: '.format(numf), fold_err)
    ad_err_cross[w] = np.mean(fold_err_cross, axis=0)
    ad_err[w] = np.mean(fold_err)
    ad_err_var[w] = np.var(fold_err)
    print('Mean of error rate: ', ad_err[w])
    print('Variance of error rate: ', ad_err_var[w])
    print('Cross error rate: ', ad_err_cross[w])
    w = w + 1

print('Results:')
print('Mean of error rate: ', ad_err)
print('Variance of error rate: ', ad_err_var)
print('Cross error rate: ', ad_err_cross)

audio: down
Fold error rate_0:  0.8901098901098901
Fold error cross rate_0:  [0.         0.15384615 0.02014652 0.02930403 0.         0.3040293
 0.1007326  0.01648352 0.24908425 0.01648352]
Fold error rate_1:  0.7767695099818511
Fold error cross rate_1:  [0.         0.01088929 0.07441016 0.47549909 0.13067151 0.
 0.0399274  0.         0.02359347 0.02177858]
Results of one audio:
2-Fold error rate:  [0.89010989 0.77676951]
Mean of error rate:  0.8334397000458706
Variance of error rate:  0.0032115104418920905
Cross error rate:  [0.         0.08236772 0.04727834 0.25240156 0.06533575 0.15201465
 0.07033    0.00824176 0.13633886 0.01913105]
audio: go
Fold error rate_0:  0.25
Fold error cross rate_0:  [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
Fold error rate_1:  0.9908759124087592
Fold error cross rate_1:  [0.04014599 0.         0.03649635 0.66788321 0.14051095 0.
 0.01094891 0.         0.08576642 0.00912409]
Results of one audio:
2-Fold error rate:  [0.25       0.99087591]
Mean of